# <font color='Teal'> **ТЕМА 2. ИНТЕЛЛЕКТУАЛЬНЫЙ АНАЛИЗ ДАННЫХ И ЕГО ПРИМЕНЕНИЕ В ЦИФРОВОМ БИЗНЕСЕ**

##<font color='Teal'> **ЛАБОРАТОРНАЯ РАБОТА 2**

Будет проведена работа с данными из социальной сети ВКонтакте через её API для анализа и кластеризации пользовательских профилей. Будет выполнен сбор данных о пользователях, включая такие аспекты, как демографическая информация, интересы, активность в группах и подписки. Эти данные пройдут процесс очистки и предобработки. Далее, на основе обработанных данных будет применена кластеризация для выявления основных групп интересов и моделей взаимодействия в сети. Результаты анализа помогут глубже понять структуру и динамику социальных связей между пользователями.

###<font size=6 color='Teal'> Подготовка среды и данных

В этом разделе производится настройка рабочего окружения и подготовка данных для последующего анализа. Настройка включает установку необходимых библиотек, подключение к Google Drive для доступа к файлам токенов и инициализацию сессии для работы с API ВКонтакте. После успешной аутентификации проводится сбор данных о друзьях пользователя, его группах и подписках, используя VK API. Данные сохраняются для дальнейшей обработки и анализа.

Устанавливаем библиотеки, необходимые для работы с VK API, например, vk-api для работы с API и requests для отправки HTTP-запросов. Это обеспечивает все технические требования для выполнения запросов к VK.

In [ ]:
!pip install vk-api

In [ ]:
import time
import pandas as pd
import seaborn as sns
import numpy as np
import math
from matplotlib import pyplot as plt
from google.colab import drive
import vk_api
import json
import ast
import re

from sklearn.preprocessing import (
    StandardScaler,
    MinMaxScaler,
    RobustScaler,
    Normalizer
    )
from sklearn.cluster import KMeans

import ipywidgets as widgets

%matplotlib inline

Используем Google Drive как хранилище для файла с токеном доступа. Монтирование позволяет непосредственно обращаться к файлам на диске из исполняемой среды.

In [ ]:
drive.mount('/content/drive')
path_to_token = '/content/drive/My Drive/vk_token.txt'

Открываем файл с токеном, который хранится на Google Drive, для аутентификации в VK API. Токен необходим для выполнения запросов от имени пользователя.

In [ ]:
with open(path_to_token, 'r') as file:
    token = file.read().strip()

Создаем сессию VK API, используя полученный токен, что позволяет делать запросы к API для извлечения данных.

In [ ]:
session = vk_api.VkApi(token=token)
vk = session.get_api()

version = 5.199

Делаем тестовый запрос к API для проверки валидности токена и доступности сервиса. Обрабатываем возможные ошибки аутентификации.

In [ ]:
try:
    response = vk.users.get()
    print(response)
except vk_api.AuthError as error_msg:
    print(error_msg)

Запрашиваем через API список друзей пользователя, выводим количество для оценки объема данных.

In [ ]:
friends = vk.friends.get(v=version)
print('Количество друзей:', friends['count'])

Получаем детальную информацию о друзьях пользователя, используя их идентификаторы. Данные включают личную информацию, образование и другие атрибуты.

In [ ]:
friends_ids_list = ",".join(map(str, friends["items"]))

fields_name = (
    "id,first_name,last_name,sex,bdate,country,home_town,"
    "has_mobile,followers_count,schools,common_count,universities,personal"
    )

print(friends_ids_list)

In [ ]:
dr = vk.users.get(
    user_ids=friends_ids_list,
    v=version,
    fields=fields_name
    )

Итеративно запрашиваем данные друзей друзей, чтобы расширить объем собираемых данных, ограничив количество до 300 человек для соблюдения ограничений API.

In [ ]:
dr_dr = []
tdr = iter(dr)

try:
    while len(dr_dr) < 300:
        user = next(tdr)
        user_friends = vk.friends.get(user_id=user["id"], v=version)['items']
        dr_dr.extend(user_friends)

        if len(dr_dr) < 300:
            time.sleep(1)
except StopIteration:
    pass

print(len(dr_dr))

Компилируем итоговый список, содержащий до 400 записей друзей и их друзей, для более глубокого анализа.

In [ ]:
m = friends["items"]
m.extend(dr_dr)
m = m[:400]
friends_ids_list = ",".join(map(str, m))

In [ ]:
dr = vk.users.get(
    user_ids=friends_ids_list,
    v=version,
    fields=fields_name
    )

In [ ]:
print(len(dr))

 Консолидируем полученные данные в DataFrame для удобства обработки и анализа.

In [ ]:
df = pd.DataFrame(dr)
df

Для каждого пользователя в DataFrame делаем запросы для получения информации о группах и подписках. Данные сохраняем в соответствующих колонках DataFrame. При этом, реализуем контроль частоты запросов, добавляя задержки, чтобы не превышать лимиты API и обрабатывая ошибки, связанные с ограничениями доступа или исключениями во время запросов.

In [ ]:
df["Группы"] = None

for index, row in df.iterrows():
    try:
        groups = vk.groups.get(
            user_id=row['id'],
            v=version,
            extended=1,
            fields="description,activity"
            )
        df.at[index, "Группы"] = json.dumps(groups["items"])
    except Exception as e:
        df.at[index, "Группы"] = None
        print(f"Ошибка: {e}")
    if (index + 1) % 10 == 0:
        print(f"Обработано {index + 1} профилей")
    time.sleep(0.3)

In [ ]:
df["Подписки"] = None

for index, row in df.iterrows():
    try:
        subscriptions = vk.users.getSubscriptions(
            user_id=row['id'],
            v=version,
            extended=1,
            fields="description,activity"
        )
        df.at[index, "Подписки"] = json.dumps(subscriptions["items"])
    except Exception as e:
        df.at[index, "Подписки"] = None
        print(f"Ошибка: {e}")
    if (index + 1) % 10 == 0:
        print(f"Обработано {index + 1} профилей")
    time.sleep(0.3)

In [ ]:
df

Сохраняем DataFrame в CSV-файл на Google Drive.

In [ ]:
df.to_csv("/content/drive/My Drive/Colab Notebooks/vk.csv", index=False)

###<font size=6 color='Teal'> Предобработка данных

В этом разделе рассматривается процесс предобработки и стандартизации данных, полученных из социальной сети ВКонтакте, чтобы они были готовы к дальнейшему анализу и интерпретации. Целью предобработки является улучшение качества данных путём исправления ошибок, заполнения пропусков и трансформации форматов данных. Эти шаги необходимы для обеспечения точности и надёжности аналитических выводов. В процесс входят: загрузка данных, переименование переменных для унификации, корректировка и стандартизация числовых и категориальных переменных, а также извлечение и анализ текстовых данных.

Загружаем данные, сохранённые в предыдущем этапе работы из файла CSV на Google Drive. Этот шаг важен для начала работы с актуальным набором данных, полученных из социальной сети.

In [ ]:
df=pd.read_csv("/content/drive/My Drive/Colab Notebooks/vk.csv")
df

Меняем названия колонок для унификации и упрощения доступа к данным в дальнейшем анализе. Это включает в себя переименование технических терминов на более понятные и удобные для работы.

In [ ]:
df.columns

In [ ]:
df = df.rename(columns={
    "bdate": "Дата рождения",
    "country": "Страна",
    'has_mobile': "Есть номер",
    'followers_count': "Подписчики",
    'common_count': "Общие друзья",
    'home_town': "Родной город",
    'schools': "Школы",
    'universities': "ВУЗы",
    'sex': "Пол",
    'first_name': "Имя",
    'last_name': "Фамилия",
    'can_access_closed': "Доступ к закрытому профилю",
    'is_closed': "Закрытый профиль",
    'deactivated': "Неактивная страница",
    'personal': "Позиция"
    }
               )

In [ ]:
df

Проанализируем и корректируем данные о дате рождения. Исключаем записи с неполными данными, выделяем год рождения, чтобы использовать его для расчёта возраста.

In [ ]:
df["Дата рождения"].str.split(".").str.len().value_counts(dropna=False)

In [ ]:
df.loc[df["Дата рождения"].str.split(".").str.len() < 3, "Дата рождения"] = None
df

In [ ]:
df["Дата рождения"] = df["Дата рождения"].str.extract(r"(\d{4})$")
df

In [ ]:
df["Дата рождения"].unique()

In [ ]:
df.loc[df["Дата рождения"].isnull(), "Дата рождения"] = None

In [ ]:
df["Дата рождения"] = df["Дата рождения"].astype('float').astype('Int64')

In [ ]:
df["Дата рождения"].unique()

In [ ]:
df.boxplot("Дата рождения")

In [ ]:
df.loc[df["Дата рождения"] < 1960, "Дата рождения"] = None

df.boxplot(column="Дата рождения")

Рассчитываем возраст на основе года рождения и анализируем распределение возраста среди пользователей через графики плотности и ящик с усами. Обрабатываем пропуски, заменяя их на нули или другие значимые значения.

In [ ]:
current_year = 2024
df["Возраст"] = current_year - df["Дата рождения"]

df = df.rename(columns={"Дата рождения": "Возраст"})

df.boxplot(column="Возраст")

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(22, 6))

sns.kdeplot(df["Возраст"], shade=True, ax=axes[0])
axes[0].set_title('Распределение плотности возраста')

sns.boxplot(y=df["Возраст"], ax=axes[1])
axes[1].set_title('Ящик с усами для возраста')

plt.show()

In [ ]:
missing_percentage = df["Возраст"].isnull().sum() / df.shape[0] * 100

print(f'Процент пропущенных значений в колонке "Возраст": {missing_percentage:.2f}%')

In [ ]:
df["Возраст"].fillna(0, inplace=True)

df.head()

Исправляем и анализируем данные о стране. Заменяем пропущенные и некорректные значения, удаляем ненужные данные, чтобы упростить модель данных.

In [ ]:
df['Страна'] = df['Страна'].fillna('{"id":-1,"title":null}').apply(lambda x: json.loads(x.replace("'", '"'))['title'])
df

In [ ]:
df['Страна'].value_counts(dropna=False)

In [ ]:
df.drop(columns=['Страна'], inplace=True)

df.head()

Удаляем данные о наличии номера телефона пользователя, так как они могут быть не релевантны для дальнейшего анализа.

In [ ]:
df['Есть номер'].value_counts(dropna=False)

In [ ]:
df.drop(columns=['Есть номер'], inplace=True)

df.head()

Анализируем количество подписчиков через графики, заполняем пропуски средним значением и преобразуем данные в целочисленный тип для удобства работы.

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(22, 6))

sns.kdeplot(df["Подписчики"], shade=True, ax=axes[0])
axes[0].set_title('График плотности для "Подписчики"')

sns.boxplot(y=df["Подписчики"], ax=axes[1])
axes[1].set_title('Диаграмма размаха (ящик с усами) для "Подписчики"')

plt.show()

In [ ]:
print(f'Доля пропущенных значений: {df["Подписчики"].isnull().sum() / df.shape[0] * 100:.2f}%')

In [ ]:
df["Подписчики"].fillna(round(df["Подписчики"].mean()), inplace=True)

df.head()

In [ ]:
df['Подписчики'] = df['Подписчики'].astype(int)

df.head()

Анализируем и обрабатываем данные о количестве общих друзей, стандартизируем и очищаем данные для точности последующих вычислений.

In [ ]:
print(f"Процент отсутствующих данных: {df['Общие друзья'].isnull().mean() * 100:.2f}%")

In [ ]:
df = df.dropna(subset=["Общие друзья"])

df

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(22, 6))

sns.kdeplot(df["Общие друзья"], shade=True, ax=axes[0])
axes[0].set_title('Распределение общих друзей')

sns.boxplot(y=df["Общие друзья"], ax=axes[1])
axes[1].set_title('Ящик с усами для общих друзей')

plt.show()

In [ ]:
df["Общие друзья"] = df["Общие друзья"].astype(int)

df

Очищаем и анализируем данные о родном городе, удаляя нерелевантные или неполные данные.

In [ ]:
print(f'Доля пропущенных значений: {df["Родной город"].isnull().mean():.2%}')

In [ ]:
df['Родной город'].value_counts(dropna=False)

In [ ]:
condition = df["Родной город"].isin(["", "Москва", "москва"]) | df["Родной город"].isnull()
print(f'Доля значений Москва или пропущенных: {condition.mean():.2%}')

In [ ]:
df.loc[~condition, "Родной город"].value_counts()

In [ ]:
df.drop(columns="Родной город", inplace=True)

df

Извлекаем и анализируем персональные данные пользователей из сложных структур JSON, включая алкоголь, вдохновение, языки и другие личные предпочтения.

In [ ]:
personal_info = df['Позиция'].fillna("{}").replace("[]", "{}").apply(ast.literal_eval).apply(pd.Series)

In [ ]:
df = df.join(personal_info[[
    'alcohol',
    'inspired_by',
    'langs',
    'life_main',
    'people_main',
    'political',
    'smoking',
    'religion',
    'religion_id'
    ]
                           ]
             )

df.rename(columns={
    'alcohol': 'Алкоголь',
    'inspired_by': 'Вдохновение',
    'langs': 'Языки',
    'life_main': 'Смысл жизни',
    'people_main': 'Главное в людях',
    'political': 'Политическая позиция',
    'smoking': 'Курение',
    'religion': 'Религия',
    'religion_id': 'id религии'},
          inplace=True
          )

df



---



In [ ]:
df['Алкоголь'] = df['Алкоголь'].replace(0, None).fillna(round(df['Алкоголь'].replace(0, None).mean()))

df



---



In [ ]:
df["Языки"].astype(str).value_counts()

In [ ]:
df.drop(columns=['Языки'], inplace=True)

df



---



In [ ]:
meanings_of_life = {
    1: "Семья и дети",
    2: "Карьера и деньги",
    3: "Развлечение и отдых",
    4: "Наука и исследования",
    5: "Совершенствование мира",
    6: "Саморазвитие",
    7: "Красота и искусство",
    8: "Слава и влияние"
    }

In [ ]:
df['Смысл жизни'] = df['Смысл жизни'].replace(0, None).replace(meanings_of_life)

df["Смысл жизни"]

In [ ]:
df["Смысл жизни"].value_counts()

In [ ]:
df["Смысл жизни"].isnull().sum()



---



In [ ]:
values_in_people = {
    1: "Ум и креативность",
    2: "Доброта и честность",
    3: "Красота и здоровье",
    4: "Власть и богатство",
    5: "Смелость и упорство",
    6: "Юмор и жизнелюбие"
    }

In [ ]:
df['Главное в людях'] = df['Главное в людях'].replace(0, None).replace(values_in_people)

df["Главное в людях"]

In [ ]:
df["Главное в людях"].isnull().sum()



---



In [ ]:
political_positions = {
    1: "Коммунизм",
    2: "Социализм",
    3: "Аполитическая",
    4: "Либерализм",
    5: "Консерватизм",
    6: "Монархия",
    7: "Ультраконсерватизм",
    8: "Аполитическая",
    9: "Либертарианство"
    }

In [ ]:
df['Политическая позиция'] = df['Политическая позиция'].replace(0, None).replace(political_positions)

df["Политическая позиция"]

In [ ]:
df["Главное в людях"].isnull().sum()



---



In [ ]:
df['Курение'] = df['Курение'].replace(0, None).fillna(round(df['Курение'].replace(0, None).mean()))

df



---



In [ ]:
df["id религии"].notnull().sum()

In [ ]:
df.drop(columns=['id религии'], inplace=True)

In [ ]:
df["Религия"].notnull().sum()

In [ ]:
df["Религия"].value_counts()



---



In [ ]:
df

In [ ]:
df.drop(columns=["ВУЗы", "Школы"], inplace=True)



---



In [ ]:
df

In [ ]:
open_profiles = df["Доступ к закрытому профилю"] != False

In [ ]:
open_profiles.sum()

In [ ]:
df = df[open_profiles]

df

In [ ]:
df.drop(columns=["Закрытый профиль", "Доступ к закрытому профилю"], inplace=True)

df



---



In [ ]:
df[df["Неактивная страница"].isnull()]

In [ ]:
df.drop(columns=["Неактивная страница"], inplace=True)

df

Производим извлечение и анализ текстовой информации из данных о группах и подписках, применяя методы обработки текста для выделения значимых слов и фраз.

In [ ]:
t = pd.DataFrame(json.loads(df["Группы"][0])).query('type == "page"')

t

In [ ]:
def extract_text_data(json_string):
    """
    Извлекает и объединяет текстовые данные из полей 'name', 'description' и 'activity' в JSON-строке.
    """
    if json_string == "[]":
        return ""
    try:
        data = pd.DataFrame(json.loads(json_string))
        pattern = r"(\w*[ЦцКкНнГгШшЩщЗзХхФфВвПпРрЛлДдЖжЧчСсМмТтБб])[УуЕеЫыАаОоЭэЯяИиЬьЮюЯяЙй]*"
        for col in ['name', 'description', 'activity']:
            data[col] = data[col].fillna("").apply(
                lambda x: " ".join(
                    x.lower() for x in re.findall(pattern, x)
                    )
                )
        data['combined'] = data['name'] + " " + data['description'] + " " + data['activity']
        return data['combined'].str.strip().sum()
    except:
        return ""

In [ ]:
df['Группы'] = df['Группы'].fillna("[]").apply(extract_text_data)
df['Подписки'] = df['Подписки'].fillna("[]").apply(extract_text_data)

In [ ]:
df

In [ ]:
def normalize_text(column):
    """
    Нормализует текст в столбце, приводя его к нижнему регистру и извлекая слова с кириллическими символами.
    """
    return " ".join(
        x.lower() for x in re.findall(
            r"(\w*[ЦцКкНнГгШшЩщЗзХхФфВвПпРрЛлДдЖжЧчСсМмТтБб])[УуЕеЫыАаОоЭэЯяИиЬьЮюЯяЙй]*",
            column.fillna("")
        )
    )

In [ ]:
columns_to_normalize = ["Вдохновение", "Смысл жизни", "Главное в людях", "Политическая позиция", "Религия"]

for column in columns_to_normalize:
    df[column] = df[column].apply(normalize_text)

df

In [ ]:
df['Текст'] = df[
    [
        'Группы',
        'Подписки',
        'Смысл жизни',
        'Главное в людях',
        'Политическая позиция',
        'Религия',
        'Вдохновение'
        ]
    ].fillna('').agg(' '.join, axis=1)

df

In [ ]:
df = df.drop(columns=[
    'Группы',
    'Подписки',
    'Смысл жизни',
    'Главное в людях',
    'Политическая позиция',
    'Религия',
    'Вдохновение'
    ]
             )

df

In [ ]:
txt = df["Текст"].sum()

In [ ]:
tdf = pd.Series(txt.split())

tdf

In [ ]:
tdf = tdf[tdf != ""]

tdf

In [ ]:
tdf = pd.DataFrame(tdf.value_counts()).reset_index().rename(columns={"index": "Слово", 0: "Частота"})

tdf

In [ ]:
tdf['Длина'] = tdf['Слово'].str.len()

tdf

In [ ]:
tdf = tdf[tdf['Длина'] > 4]

tdf

In [ ]:
tdf["Слово"][:50]

In [ ]:
keywords = ["музык", "москв", "работ", "новост", "фотограф", "творчеств", "реклам", "бизнес", "магазин", "культур"]
categories = ["Музыка", "Москва", "Работа", "Новости", "Фотография", "Творчество", "Реклама", "Бизнес", "Магазин", "Культура"]

In [ ]:
def calculate_frequencies(text):
    """
    Вычисляет частоту встречаемости ключевых слов в тексте.
    """
    words = pd.Series(text.split())
    filtered_words = words[words != ""]
    word_counts = filtered_words.value_counts().reindex(keywords, fill_value=0)
    return word_counts

In [ ]:
frequencies = df['Текст'].apply(calculate_frequencies)
df[categories] = frequencies[keywords].values.tolist()

df

In [ ]:
df = df.drop(columns=["Текст"])

df

In [ ]:
df = df.set_index("id")

df

In [ ]:
df[df.isna().any(axis=1)]

In [ ]:
df.loc[4044219]

In [ ]:
categories = ["Музыка", "Москва", "Работа", "Новости", "Фотография", "Творчество", "Реклама", "Бизнес", "Магазин", "Культура"]

In [ ]:
sum_categories = df[categories].fillna(0).sum(axis=1)
df[categories] = df[categories].div(sum_categories, axis=0) * 100
df[categories] = df[categories].round()

df

In [ ]:
df = df.dropna()

df

###<font size=6 color='Teal'> Кластеризация

Теперь сосредоточимся на применении методов машинного обучения для группировки данных — процессе, известном как кластеризация. Цель данного процесса — идентифицировать неявные подгруппы в наборе данных, состоящем из пользовательских профилей, исходя из их интересов и других особенностей.

 Сначала создадим копию текущего набора данных для сохранности исходной информации. Затем исключим из анализа такие атрибуты, как имя, фамилия, количество подписчиков и общих друзей, поскольку они не несут значимой информации для задачи кластеризации.

In [ ]:
old_df = df.copy(deep=True)

In [ ]:
df.drop(columns=["Имя", "Фамилия", "Подписчики", "Общие друзья"], inplace=True)

df

Следующий шаг — стандартизация данных. Поскольку алгоритмы машинного обучения чувствительны к масштабу переменных, используем StandardScaler для нормализации значений. Это обеспечит более качественное и корректное разделение данных на группы.

In [ ]:
scalers = {col: StandardScaler() for col in df.columns}

for col, scaler in scalers.items():
    df[col] = scaler.fit_transform(df[[col]])

df

Используя метод `corr()`, получаем корреляционную матрицу, которая представляет коэффициенты Пирсона, варьирующиеся от –1 до 1, для оценки линейных взаимосвязей между переменными; это необходимо для выявления сильных связей, определения избыточности переменных и изучения мультиколлинеарности, что важно для точности интерпретации кластеров и последующего аналитического применения результатов.

Коэффициенты корреляции могут варьироваться от –1 до 1, где:

* 1 означает положительную линейную зависимость (когда одна переменная увеличивается, другая тоже увеличивается),

* –1 означает отрицательную линейную зависимость (одна переменная увеличивается, другая уменьшается),

* 0 указывает на отсутствие линейной зависимости между переменными.

In [ ]:
df.corr()

После подготовки данных применим алгоритм `KMeans` для разделения пользователей на группы (в данном случае на пять групп).

In [ ]:
model = KMeans(n_clusters=5)
model.fit(df)

Полученные метки кластеров добавим к исходному датафрейму, что позволит проанализировать средние значения переменных в каждом кластере и визуализировать распределение ключевых интересов в виде столбчатой диаграммы. Это даст представление о характерных особенностях каждой группы.

In [ ]:
old_df["Группа"] = model.labels_

old_df

In [ ]:
mean_df = old_df.groupby('Группа').mean()

mean_df

In [ ]:
mean_df[
    [
        "Музыка", "Москва", "Работа", "Новости", "Фотография", "Творчество",
        "Реклама", "Бизнес", "Магазин", "Культура"
        ]
    ].reset_index().plot.bar(
        x="Группа",
        stacked=True,
        figsize=(10, 10)
        )

Наконец, проведём интерпретацию кластеров, переименовав их в соответствии с выявленными характерными чертами и обогатим анализ преобразованием числовых значений пола в категориальные для более удобного восприятия.

In [ ]:
old_df.loc[old_df["Пол"] == 1, "Пол"] = "Женский"
old_df.loc[old_df["Пол"] == 2, "Пол"] = "Мужской"

In [ ]:
clusters = {
    "Новости и реклама": old_df[old_df["Группа"] == 0],
    "Москва": old_df[old_df["Группа"] == 1],
    "Музыка": old_df[old_df["Группа"] == 2],
    "Фотография и творчество": old_df[old_df["Группа"] == 3],
    "Музыка и Москва": old_df[old_df["Группа"] == 4]
    }

###<font size=6 color='Teal'> Подбор целевой аудитории

Сфокусируемся на подборе целевой аудитории с использованием результатов кластеризации. Для этого реализуем интерактивный интерфейс, который позволит выбрать конкретные группы пользователей на основе их интересов, возраста и пола. Используем выпадающие списки (`widgets.Dropdown`), каждый из которых предоставит различные опции для фильтрации данных.

По нажатию на кнопку «Сформировать список» будет активирована функция `filter_data()`, которая отфильтрует набор данных согласно выбранным параметрам. Результатом фильтрации будут списки пользователей с именами, фамилиями, возрастом и полом, а также их уникальные идентификаторы, которые затем можно использовать, например, в маркетинговых или аналитических целях.

Такой подход позволит оперативно адаптировать подбор аудитории под конкретные задачи и упростит процесс выделения подгрупп пользователей для дальнейших действий.

In [ ]:
interests_dropdown = widgets.Dropdown(
    options=clusters.keys(),
    value=list(clusters.keys())[0],
    description="Интересы:"
    )

age_dropdown = widgets.Dropdown(
    options=["Любой", "Только 18+"],
    value="Любой",
    description="Возраст:"
    )

gender_dropdown = widgets.Dropdown(
    options=["Любой", "Мужской", "Женский"],
    value="Любой",
    description="Пол:"
    )

generate_button = widgets.Button(description='Сформировать список')

def filter_data(_):
    selected_cluster = clusters[interests_dropdown.value]
    if age_dropdown.value == "Только 18+":
        selected_cluster = selected_cluster[selected_cluster["Возраст"] >= 18]
    if gender_dropdown.value != "Любой":
        selected_cluster = selected_cluster[selected_cluster["Пол"] == gender_dropdown.value]
    print(selected_cluster[["Имя", "Фамилия", "Возраст", "Пол"]])
    print(list(selected_cluster.index))

generate_button.on_click(filter_data)

widgets.VBox([interests_dropdown, age_dropdown, gender_dropdown, generate_button])

##<font color='Teal'> **ЗАДАНИЕ**

**Цель задания:**

Овладеть методикой сбора данных из социальной сети vk.com, провести их предобработку и анализ с целью выявления типичных предпочтений пользователей, и разработать программное решение для автоматизированного навязывания рекомендаций для определенных кластеров.

**Задачи**:

1. **Сбор данных**:
   - Используя API социальной сети vk.com, извлеките необходимый массив данных о пользователях, включая личную информацию и данные о группах и предпочтениях.
   - Обеспечьте соблюдение правил сбора данных и этических норм.
   - Оформите промежуточный отчёт о результатах сбора данных, включая описание используемых методов и столкнувшихся проблем.

2. **Предобработка данных**:
   - Проведите очистку, нормализацию и трансформацию собранных данных.
   - Разработайте алгоритмы для корректной обработки исключений и пропущенных значений.
   - Зафиксируйте в отчёте этапы предобработки данных и любые значимые наблюдения.

3. **Кластерный анализ**:
   - Примените методы машинного обучения для выявления кластеров пользователей по их предпочтениям.
   - Осуществите статистический и визуальный анализ характеристик кластеров.
   - Подготовьте раздел отчёта с описанием выбранных алгоритмов, полученных результатов и анализом характеристик кластеров.

4. **Разработка автоматизированной системы рекомендаций**:
   - Спроектируйте и реализуйте веб-приложение с использованием фреймворка Flask, которое будет рекомендовать контент для выбранных кластеров пользователей.
   - Разработайте алгоритмы для персонализированных рекомендаций, основываясь на профиле пользователя и его принадлежности к определенному кластеру.
   - Включите в отчёт детали разработки системы, описание архитектуры веб-приложения и примеры работы рекомендательного сервиса.

5. **Подготовка отчета**:
   - Составьте итоговый отчет о проделанной работе, который включает в себя введение, методологию, результаты, обсуждение и выводы.
   - Отчет должен отражать всю проделанную работу, аналитические подходы, результаты кластерного анализа, принципы работы разработанной системы и оценку её эффективности.
   - Представьте рекомендации по дальнейшему использованию и развитию системы.

**Ожидаемый результат**:

Разработанная система должна автоматизировать процесс рекомендации контента, учитывая различные интересы и предпочтения пользователей, и способствовать улучшению целевой аудитории рекламных и информационных кампаний. Работа включает в себя не только аналитическую часть, но и разработку полноценного программного продукта с веб-интерфейсом. Финальный отчет должен подробно отображать весь исследовательский и разработочный процессы, а также содержать критический анализ и предложения по улучшению системы.